In [1]:
import os
import gymnasium as gym
import numpy as np

import ray
from ray.tune.registry import register_env
from ray.rllib.algorithms import ppo, dqn, appo, impala
from ray.rllib.policy.policy import PolicySpec

2023-08-15 11:06:28,529	WARNING deprecation.py:50 -- DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!
2023-08-15 11:06:28,679	WARNING deprecation.py:50 -- DeprecationWarning: `build_tf_policy` has been deprecated. This will raise an error in the future!
2023-08-15 11:06:28,686	WARNING deprecation.py:50 -- DeprecationWarning: `build_policy_class` has been deprecated. This will raise an error in the future!


In [2]:
from rl.envs.othello import OthelloEnv
from rl.agents.random_policy import OthelloRandomPolicy

In [3]:
ray.init(include_dashboard=False)

2023-08-15 11:06:31,337	INFO worker.py:1621 -- Started a local Ray instance.


Python version:,3.8.17
Ray version:,2.6.2


In [4]:
register_env("othello", lambda _: OthelloEnv({}))

In [5]:
model_config = {
    "fcnet_hiddens": [512, 512],
}

def policy_mapping_fn(agent_id, episode, worker, **kwargs):
    agent_id = int(agent_id[-1]) - 1
    return "agent_1" if episode.episode_id % 2 == agent_id else "agent_2"

config = ppo.PPOConfig().environment("othello").framework("torch").rollouts(num_rollout_workers=os.cpu_count() // 2).resources(num_gpus=0)
config = config.multi_agent(policies={"agent_1": PolicySpec(), "agent_2": PolicySpec(policy_class=OthelloRandomPolicy)}, policy_mapping_fn=policy_mapping_fn, policies_to_train=["agent_1"])
config = config.training(model=model_config, _enable_learner_api=False)
config = config.rl_module(_enable_rl_module_api=False)
# config = config.training(num_sgd_iter=10, model={"conv_filters": [[32, [3, 3], 1], [64, [3, 3], 1]]})
# config = config.training(model={"conv_filters": [[32, [3, 3], 2], [64, [3, 3], 2]]}, _enable_learner_api=False)

2023-08-15 11:06:33,585	WARNING algorithm_config.py:2558 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_config={}`.
2023-08-15 11:06:33,586	WARNING algorithm_config.py:2572 -- Setting `exploration_config={'type': 'StochasticSampling'}` because you set `_enable_rl_module_api=False`. This exploration config was restored from a prior exploration config that was overriden when setting `_enable_rl_module_api=True`. This occurs because when RLModule API are enabled, exploration_config can not be set.


In [6]:
# import random
# # multiagent environment for othello
# env = OthelloEnv({})
# obs, _ = env.reset()
# terminated = False
# current_player = "agent_1"
# while not terminated:
#     env.render()
#     # action = env.action_space.sample()
#     valid_actions = env.get_valid_moves(current_player)
#     print(current_player, obs.keys())
#     print(obs[current_player].shape)
#     action = algo.compute_single_action(obs[current_player], policy_id=current_player)
#     print(action)
#     if len(valid_actions) > 0:
#         action = random.choice(valid_actions)
#     else:
#         action = 64
#     action = {current_player: action}
#     obs, reward, terminated, truncated, _= env.step(action)
#     reward = reward[current_player]
#     terminated = terminated[current_player]
#     truncated = truncated[current_player]
#     current_player = env.current_player
#     print(terminated, truncated, reward)

In [7]:
from ray import air
from ray import tune

results = tune.Tuner(
    "PPO",
    param_space=config,
    run_config=air.RunConfig(
        checkpoint_config=air.CheckpointConfig(
            checkpoint_at_end=True,
            checkpoint_frequency=50,
        )
    ),
).fit()

2023-08-15 11:06:33,832	WARNING algorithm_config.py:2558 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_config={}`.
2023-08-15 11:06:33,833	WARNING algorithm_config.py:2572 -- Setting `exploration_config={'type': 'StochasticSampling'}` because you set `_enable_rl_module_api=False`. This exploration config was restored from a prior exploration config that was overriden when setting `_enable_rl_module_api=True`. This occurs because when RLModule API are enabled, exploration_config can not be set.
(pid=12876) DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!
(PPO pid=12876) 2023-08-15 11:06:38,481	WARNING algorithm_config

In [8]:
results

ResultGrid<[
  Result(
    metrics={'custom_metrics': {}, 'episode_media': {}, 'info': {'learner': {'agent_1': {'learner_stats': {'allreduce_latency': 0.0, 'grad_gnorm': 2.3644271208594243, 'cur_kl_coeff': 0.7593750000000002, 'cur_lr': 5.0000000000000016e-05, 'total_loss': 8.325176572799682, 'policy_loss': -0.062123337705270384, 'vf_loss': 8.380824494361878, 'vf_explained_var': 0.016079674288630486, 'kl': 0.008527369575162902, 'entropy': 0.627846292903026, 'entropy_coeff': 0.0}, 'model': {}, 'custom_metrics': {}, 'num_agent_steps_trained': 125.0625, 'num_grad_updates_lifetime': 297840.5, 'diff_num_grad_updates_vs_sampler_policy': 239.5}}, 'num_env_steps_sampled': 2484000, 'num_env_steps_trained': 2484000, 'num_agent_steps_sampled': 2483992, 'num_agent_steps_trained': 2483992}, 'sampler_results': {'episode_reward_max': 53.0, 'episode_reward_min': -15.0, 'episode_reward_mean': -2.905861456483126, 'episode_len_mean': 7.147424511545293, 'episode_media': {}, 'episodes_this_iter': 563, 'poli

In [9]:
checkpoint = results.get_best_result().checkpoint
print(checkpoint)

Checkpoint(uri=c://\Users\nYoshiaki\ray_results\PPO\PPO_othello_5ba33_00000_0_2023-08-15_11-06-33\checkpoint_000600)


In [10]:
import ray
ray.shutdown()
register_env("othello", lambda _: OthelloEnv({}))

In [11]:
config.expolore = False
algo = config.build()
algo.restore(checkpoint)

2023-08-15 13:00:38,989	WARNING algorithm_config.py:2558 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_config={}`.
c:\Users\nYoshiaki\AppData\Local\miniconda3\envs\rl\lib\site-packages\ray\rllib\algorithms\algorithm.py:484: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
`UnifiedLogger` will be removed in Ray 2.7.
  return UnifiedLogger(config, logdir, loggers=None)
c:\Users\nYoshiaki\AppData\Local\miniconda3\envs\rl\lib\site-packages\ray\tune\logger\unified.py:53: RayDeprecationWarning: This API is deprecated and may be removed in

In [13]:
algo.get_policy("agent_1").export_model("othello_policy", onnx=18)

============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



In [14]:
import random
# multiagent environment for othello
env = OthelloEnv({})
obs, _ = env.reset()
terminated = False
current_player = "agent_1"
while not terminated:
    env.render()
    # action = env.action_space.sample()
    valid_actions = env.get_valid_moves(current_player)
    action = algo.compute_single_action(obs[current_player], policy_id=current_player)
    # if len(valid_actions) > 0:
    #     action = random.choice(valid_actions)
    # else:
    #     action = 64
    action = {current_player: action}
    print(action)
    obs, reward, terminated, truncated, _= env.step(action)
    reward = reward[current_player]
    terminated = terminated[current_player]
    truncated = truncated[current_player]
    current_player = env.current_player

.|.|.|.|.|.|.|.
.|.|.|.|.|.|.|.
.|.|.|.|.|.|.|.
.|.|.|O|X|.|.|.
.|.|.|X|O|.|.|.
.|.|.|.|.|.|.|.
.|.|.|.|.|.|.|.
.|.|.|.|.|.|.|.

{'agent_1': 29}
.|.|.|.|.|.|.|.
.|.|.|.|.|.|.|.
.|.|.|.|.|.|.|.
.|.|.|O|O|O|.|.
.|.|.|X|O|.|.|.
.|.|.|.|.|.|.|.
.|.|.|.|.|.|.|.
.|.|.|.|.|.|.|.



TypeError: object of type 'int' has no len()

In [ ]:
env.render()

.|.|.|.|.|.|.|.
.|.|.|.|.|.|.|.
.|.|.|.|.|.|.|.
.|.|.|O|O|O|.|.
.|.|.|X|X|X|.|.
.|.|.|.|.|.|.|.
.|.|.|.|.|.|.|.
.|.|.|.|.|.|.|.

